# Part-3: Baseline Validation

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для самопроверки: у Сергея средняя корреляция Спирмена на тестовом множестве 2020 года во всех моделях, включая baselines, получалась порядка 0.7-0.8, а корреляция Кендалла — порядка 0.5-0.6. Если у корреляции вышли за 0.9 или, наоборот, упали ниже 0.3, скорее всего где-то баг.

In [41]:
import pickle
import numpy as np
from scipy import stats

In [8]:
# source: https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

#### * Вес (сила) команды = вес ее участников (их обучили на трейне)
#### * В рамках одного турнира предлагается ранжировать команды по их весу

In [6]:
player_to_weight = load_obj('player_to_weight')

In [9]:
df_test = load_obj('test')

In [40]:
def get_positions_label(tournament):
    return [team['position'] for team in tournament]

def get_position_prediction(tournament):
    """
    ранжируем команды по весу = (сумма весов участников),
    есть игрока не было в train -- берем средний вес игрока в трейне
    """
    avg_weight = np.mean([v for v in player_to_weight.values()])
    team_rating = []
    for idx, team in enumerate(tournament):
        weight = 0
        for player_info in team['teamMembers']:
            p_id = player_info['player']['id']
            try:
                weight += player_to_weight[p_id]
            except:
                weight += avg_weight
        team_rating.append((idx + 1, weight))
    team_rating = sorted(team_rating, key=lambda kv: kv[1], reverse=True)
    return [pos for pos, weight in team_rating]

In [65]:
def get_score(df, corr):
    x = [corr(get_positions_label(t), get_position_prediction(t)).correlation for t in df.values()]
    x = np.array(x)
    x = x[~np.isnan(x)]
    return np.mean(x)

for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
    print(f'Avg {corr[0]} corr value for df = {get_score(df_test, corr[1])}')

Avg Spearman corr value for df = 0.7821848129235908
Avg Kendall  corr value for df = 0.6252786821846475
